In [37]:
from keras.models import load_model
from keras.layers import Dense, Flatten
from keras import Model
import pandas as pd

In [38]:
model = load_model('pickles/train_3_66.7-emb_dr3_bi_dr5_bi_dr4.h5')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 32, 300)           11090400  
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 300)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 300)           541200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 300)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 300)           541200    
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 300)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9600)              0         
__________

In [52]:
def reshape_model(model):
    model.pop()
    model.pop()
    for layer in model.layers:
        layer.trainable = False

    dense1 = Dense(150, activation='relu')
    dense2 = Dense(7, activation='softmax')

    model.add(dense1)
    model.add(Flatten())
    model.add(dense2)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [53]:
import pickle

In [54]:
model = load_model('./pickles/train_3_67.2-emb_dr3_bi_dr5_bi_dr4.h5')

In [55]:
reshape_model(model)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 32, 342)           12643056  
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 342)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 300)           591600    
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 300)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 300)           541200    
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 300)           0         
_________________________________________________________________
dense_7 (Dense)              (None, 32, 150)           45150     
__________

In [56]:
tweets = pd.read_feather('pickles/train_7.feather')
tweets.head(10)

,ID,Class,Tweet,Sentiment
0,0,0,yeah ☺ ️ playing well,0
1,1,0,least not guy try discourage anymore want neve...,0
2,2,0,uplift still discourage mean listen wrong voic...,0
3,3,0,... age heyday blood tame ...,0
4,4,-2,embarrassed saw u like knvfkkjg think stalker ...,-2
5,5,-3,really plan make video week tv die phone break...,-3
6,6,-3,hate idea afraid share,-3
7,7,1,regular cheerfulness emotion supper give lady ...,1
8,8,1,pessimist see difficulty every opportunity opt...,1
9,9,-2,hurt ndoes not mean hurt ndoes not mean not ge...,-2


In [57]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

In [58]:
train_tweets, sentiments = tweets['Tweet'], tweets['Sentiment']

tokenizer = Tokenizer(filters=' ')
tokenizer.fit_on_texts(train_tweets)
sequences = tokenizer.texts_to_sequences(train_tweets)

MAX_SEQUENCE_LENGTH = 0
for elt in sequences:
    if len(elt) > MAX_SEQUENCE_LENGTH:
        MAX_SEQUENCE_LENGTH = len(elt)

train_sequences = pad_sequences(train_sequences, 32)
targets = to_categorical(sentiments, 7)

In [59]:
print(train_sequences.shape)
print(targets.shape)

(1630, 32)
(1630, 7)


In [60]:
X_train, X_val, y_train, y_val = train_test_split(train_sequences, targets, test_size=0.3)

print('training set: ' + str(len(X_train)) + ' samples')
print('validation set: ' + str(len(X_val)) + ' samples')

print('x_train:', X_train.shape)
print('y_train:', y_train.shape)


training set: 1141 samples
validation set: 489 samples
x_train: (1141, 32)
y_train: (1141, 7)


In [62]:
model.fit(X_train, y_train, batch_size=128, validation_data=(X_val, y_val), epochs=5)

Train on 1141 samples, validate on 489 samples
Epoch 1/5
1141/1141 [==============================] - 9s 8ms/step - loss: 1.8014 - acc: 0.2857 - val_loss: 1.8699 - val_acc: 0.2454
Epoch 2/5
1141/1141 [==============================] - 9s 8ms/step - loss: 1.7762 - acc: 0.2971 - val_loss: 1.8707 - val_acc: 0.2372
Epoch 3/5
1141/1141 [==============================] - 9s 8ms/step - loss: 1.7471 - acc: 0.2971 - val_loss: 1.8723 - val_acc: 0.2352
Epoch 4/5
1141/1141 [==============================] - 9s 8ms/step - loss: 1.6970 - acc: 0.3181 - val_loss: 1.8985 - val_acc: 0.2331
Epoch 5/5
1141/1141 [==============================] - 10s 8ms/step - loss: 1.6239 - acc: 0.3497 - val_loss: 1.9601 - val_acc: 0.2454
